In [1]:
import pyarrow.parquet as pq
import pandas as pd
import polars as pl
import datetime
from polars.datatypes import (
    Boolean, UInt8, UInt16, UInt32, UInt64, Utf8,
    Int8, Int16, Int32, Int64,
    Float32, Float64
)
from polars.datatypes import Utf8, Datetime, Date, Time
import xgboost as xgb
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold
from sklearn.metrics import ndcg_score
from geopy.distance import geodesic
from src.utils.utils import evaluate_feature
from src.train_new_base import train_base_model
rows = (
    pl.scan_parquet("data/train.parquet")        # or scan_parquet/scan_ndjson/…
      .select(pl.len())           # len() == COUNT(*)
      .collect(streaming=True)    # constant-memory execution
      .item()                     # get the scalar
)

C:\Users\1tiii\AppData\Local\Temp\ipykernel_23220\103309096.py:22: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .collect(streaming=True)    # constant-memory execution


In [2]:
data = pl.read_parquet("data/clean_data_cut.parquet")

In [14]:
feature_cols = ['company_count', 'company_freq', 'totalPrice', 'route_distance_km', 'frequentFlyer_n_programs', 'unique_ranker_count', 'hour_sin', 
                'hour_cos', 'wday_sin', 'wday_cos', 'twoway_route', 'avg_oneway_price', 'total_duration_hours_leg0', 'total_duration_hours_leg1', 'legs0_segments0_aircraft_code_freq',
                'legs1_segments0_aircraft_code_freq', 'frequentFlyer', 'isVip', 'miniRules0_monetaryAmount', 'miniRules0_percentage',
                'miniRules1_monetaryAmount', 'miniRules1_percentage', 'sex', 'total_segments_count', 'legs0_num_segments', 'legs1_num_segments', 'tariff_code_filled', 'nationality_cat', 'miniRules0_statusInfos',
                'miniRules1_statusInfos', 'pricingInfo_isAccessTP', 'legs0_departureAt_period', 'legs0_arrivalAt_period', 'legs1_departureAt_period', 'legs1_arrivalAt_period',
                'legs0_segments0_baggageAllowance_quantity_kg', 'legs1_segments0_baggageAllowance_quantity_kg', 'legs0_segments0_cabinClass_cat', 'legs1_segments0_cabinClass_cat']
FEATURE_COLUMNS_V1 = [
    # 1-2. Пересадки
    "layover_hours_leg0",
    "layover_hours_leg1",

    # 3. Сколько часов до вылёта
    "days_before_flight_leg0",

    # 4 + 6. Смена аэропорта внутри одной ноги
    "legs0_airport_changes_count",
    "legs1_airport_changes_count",

    # 5. Ночная пересадка
    "night_layover_leg0",
    "night_layover_leg1",

    # 7 + 11. Одна ли авиакомпания-оператор
    "same_operator_carrier_leg0",
    "same_operator_carrier_leg1",

    # 8. Участие билета в программе лояльности
    "ticket_is_in_FFprogramms_leg0",
    "ticket_is_in_FFprogramms_leg1",

    # 9. Ранг по цене
    "totalPrice_rank",

    # 10. Ранг по общему времени перелёта
    "totalTime_hours_ranked",

    # 12. Согласованность багажа
    "baggage_kg_equal_flag",
    "baggage_units_equal_flag",

    # 13. Сколько билетов показано в поисковой сессии
    "tickets_in_session",

    # 14. Метрики оставшихся билетов
    "remainingTickets_avg",
    "remainingTickets_rank",

    # 15. Частота покупок пользователя
    "user_search_freq",

    # 16. Доля совпадений оператор-продавец
    "operator_marketer_match_rate",

    # 17. Билет ≤ 20 % дороже минимального
    "within_20pct_of_min",

    # 18. День недели вылета (синус/косинус)
    "leg0_depday_sin", "leg1_depday_sin",
    "leg0_depday_cos", "leg1_depday_cos",

    # 19. День недели прилёта (синус/косинус)
    "leg0_arrday_sin", "leg1_arrday_sin",
    "leg0_arrday_cos", "leg1_arrday_cos",

    # 20. Комплексная «оптимальность» билета
    "opt_ticket_score",
]
FEATURE_COLUMNS_V2 = [
    # Cчётчики сегментов и направления
    "n_segments_leg0",
    "n_segments_leg1",
    "is_one_way",
    "total_segments",
    "is_direct_leg0",
    "is_direct_leg1",
    "both_direct",

    # Стоимость, налоги, сборы
    "price_per_tax",
    "tax_rate",
    "log_price",
    "total_fees",
    "has_fees",
    "fee_rate",

    # Продолжительности
    "duration_ratio",

    # Тарифы и привилегии
    "has_corporate_tariff",
    "has_access_tp",
    "n_ff_programs",
    "is_vip_freq",

    # Класс обслуживания
    "avg_cabin_class",
    "cabin_class_diff",

    # Популярные маршруты
    "is_popular_route",

    # Относительные метрики цены и сегментов внутри поиска
    "price_pct_rank",
    "is_cheapest",
    "price_from_median",
    "is_min_segments",
    "is_direct_cheapest",
]
feature_list = feature_cols + FEATURE_COLUMNS_V1 + FEATURE_COLUMNS_V2
cat_features_final = ['tariff_code_filled', 'nationality_cat', 'legs0_departureAt_period', 'legs0_arrivalAt_period',
                      'legs1_departureAt_period', 'legs1_arrivalAt_period', 'legs0_segments0_cabinClass_cat', 'legs1_segments0_cabinClass_cat', 'frequentFlyer']
#feature_cols = ['totalPrice']
#cat_features_final = []
#X = data.select(feature_cols)
#y = data.select("selected")            # Polars DataFrame with 1 col
#groups = data.select("ranker_id")      # Polars DataFrame with 1 col
n2 = rows

# 5. Set params to use pairwise ranking + ndcg@3 + histogram tree builder
params = {
    "objective":   "rank:pairwise",
    "eval_metric": "ndcg@3",
    "tree_method": "hist",      # 🔥 much faster
    "seed":        42,
    "n_jobs":      -1,
    'eta': 0.14381590394747168,
    "learning_rate": 0.022641389657079056,
    'max_depth': 14,
    'min_child_weight': 2,
    'subsample': 0.7308722227055789,
    "colsample_bytree": 0.45840689146263086,
    "gamma": 3.3084297630544888,
    "lambda": 6.952586917313028,
    "alpha": 0.6395254133055179,
    'num_boost_round': 1500
}


In [5]:
cv_stats = pd.read_csv('model/cv_results.csv')
print('mean top@3 - ', round(cv_stats['val-top@3'].mean(), 4))
print('std top@3 - ', round(cv_stats['val-top@3'].std(), 4))
print('mean ndcg@3 - ', round(cv_stats['val-ndcg@3'].mean(), 4))
print('std ndcg@3 - ', round(cv_stats['val-ndcg@3'].std(), 4))

mean top@3 -  0.4192
std top@3 -  0.0027
mean ndcg@3 -  0.3848
std ndcg@3 -  0.002


In [5]:
feature_list = ['totalPrice', 'ranker_id', 'selected', 'company_count', 'route_distance_km', 'frequentFlyer_n_programs', 'unique_ranker_count']

In [ ]:
top_sigma = np.std(pd.read_csv('model/cv_results.csv')['val-top@3']), np.mean(pd.read_csv('model/cv_results.csv')['val-top@3'])
meta, df = evaluate_feature(data[:n2][feature_list], 
                       new_feature_name='unique_ranker_count', 
                       group_col='ranker_id', 
                       label_col='selected', 
                       baseline_model='model/base.json', 
                       params=params, 
                       sigma0=top_sigma[1],
                       corr_threshold=0.00,
                       cat_features_final=cat_features_final)
meta

top@3 before -  0.2258  top@3 after -  0.2237
ndcg@3 before -  0.2622  top@3 after -  0.2614
mean top@3 -  0.2239
std top@3 -  0.003
mean ndcg@3 -  0.2621
std ndcg@3 -  0.0012


{'corr': np.float64(0.005832222368000487),
 'warm_delta': np.float64(-0.0021618937392779147),
 'mini_delta': None}

In [47]:
if round(cv_stats['val-top@3'].mean(), 4) < round(df['val-top@3'].mean(), 4):
    print('accepted by top@3')
if round(cv_stats['val-ndcg@3'].mean(), 4)  < round(df['val-ndcg@3'].mean(), 4):
    print('accepted by ncdg3')

accepted by ncdg3


In [43]:
df.to_pandas().to_csv('model/cv_results.csv')

In [ ]:
res = train_base_model(data=data[:n2], 
                 features=feature_list, 
                 label_col='selected', 
                 group_col='ranker_id', 
                 params=params, 
                 num_boost_round=1000, 
                 baseline_model_path='model/', 
                 seed=42, 
                 verbose_eval_size=5, 
                 full_cv = False,
                 cat_features_final=cat_features_final)

res

d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:14:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "num_boost_round" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0.4694783687591553


In [6]:
booster = xgb.Booster()
booster.load_model('model/base.json')

In [9]:
def get_group_sizes(ranker_ids: np.ndarray) -> np.ndarray:
    # unique in order of first appearance + their counts
    uniq, idx, counts = np.unique(ranker_ids, return_index=True, return_counts=True)
    return counts[np.argsort(idx)]
data_xgb = data[n2:].with_columns([
                                (pl.col(c).rank("dense") - 1)
                                .fill_null(-1)
                                .cast(pl.Int32)
                                .alias(c)
                                for c in cat_features_final
                            ])
label_col = 'selected'
group_col = 'ranker_id'
feature_cols_all = feature_list
data_test = xgb.DMatrix(data_xgb.select(feature_cols_all).to_numpy(), feature_names=feature_cols_all, group=get_group_sizes(data_xgb.select(group_col)))

In [10]:
preds = booster.predict(data_test)

In [11]:
test = pl.read_parquet('data/test.parquet')

In [12]:
submission_xgb = (
    test.select(['Id', 'ranker_id'])
    .with_columns(pl.Series('pred_score', preds))
    .with_columns(
        pl.col('pred_score')
        .rank(method='ordinal', descending=True)
        .over('ranker_id')
        .cast(pl.Int32)
        .alias('selected')
    )
    .select(['Id', 'ranker_id', 'selected', 'pred_score'])
)

In [13]:
submission_xgb.write_csv('submission_v2.csv')

In [ ]:
import optuna
LABEL_COL = "selected"          # binary 1/0 – chosen by the user
GROUP_COL = "ranker_id"         # search/session id
FEATURE_COLUMNS = feature_list
CAT_FEATURES_FINAL = cat_features_final
def get_group_sizes(ranker_ids: np.ndarray) -> np.ndarray:
    """Return group‐sizes array for XGBoost ranking.

    Parameters
    ----------
    ranker_ids : np.ndarray
        1‑D array with group id (ranker_id) of every row in the same order as X.

    Returns
    -------
    np.ndarray
        Array of group sizes, ordered by the first appearance of each id (exactly
        what xgb.DMatrix wants for ranking).
    """
    uniq, idx, counts = np.unique(ranker_ids, return_index=True, return_counts=True)
    return counts[np.argsort(idx)]
def train_single_split(
    data_xgb: pl.DataFrame,
    params: dict,
    num_boost_round: int,
    seed: int = 42,
) -> float:
    """Train XGBoost on a random 80/20 group‑based split and return hit@3."""

    # Ordinal‑encode categorical features ------------------------------------------------


    rng = np.random.default_rng(seed)
    groups_unique = data_xgb.select(GROUP_COL).unique().to_numpy().ravel()
    val_groups = rng.choice(groups_unique, size=max(1, int(0.2 * len(groups_unique))), replace=False)

    val_df = data_xgb.filter(pl.col(GROUP_COL).is_in(val_groups))
    train_df = data_xgb.filter(~pl.col(GROUP_COL).is_in(val_groups))

    feature_cols = [c for c in FEATURE_COLUMNS if c not in {LABEL_COL, GROUP_COL}]

    dtrain = xgb.DMatrix(
        train_df.select(feature_cols).to_numpy(),
        label=train_df.select(LABEL_COL).to_numpy().ravel(),
        group=get_group_sizes(train_df.select(GROUP_COL).to_numpy().ravel()),
        feature_names=feature_cols,
    )

    dval = xgb.DMatrix(
        val_df.select(feature_cols).to_numpy(),
        label=val_df.select(LABEL_COL).to_numpy().ravel(),
        group=get_group_sizes(val_df.select(GROUP_COL).to_numpy().ravel()),
        feature_names=feature_cols,
    )

    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=[(dval, "val")],
        early_stopping_rounds=30,
        verbose_eval=False,
    )

    # NDCG@3 produced by XGBoost is geometric mean across groups; for hit@3 we compute manually
    preds = bst.predict(dval)

    # Hit@3 – at least one correct item in top‑3 per group ------------------
    # Reconstruct group slices
    ranker_ids = val_df.select(GROUP_COL).to_numpy().ravel()
    labels = dval.get_label()
    order = np.argsort(ranker_ids, kind="stable")
    ranker_ids, labels, preds = ranker_ids[order], labels[order], preds[order]

    hit_total, group_total = 0, 0
    start = 0
    while start < len(ranker_ids):
        group = ranker_ids[start]
        end = start
        while end < len(ranker_ids) and ranker_ids[end] == group:
            end += 1
        if (end - start) > 10:
            group_labels = labels[start:end]
            group_preds = preds[start:end]
            top3_idx = np.argsort(group_preds)[::-1][:3]
            hit_total += int(group_labels[top3_idx].sum() > 0)
            group_total += 1
        start = end
    return hit_total / group_total


# ---------------------------------------------------------------------------
# Optuna objective ----------------------------------------------------------
# ---------------------------------------------------------------------------

def objective(trial: optuna.Trial, data: pl.DataFrame) -> float:
    params = {
        "objective": "rank:pairwise",
        "eval_metric": "ndcg@3",
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
        "tree_method": "hist",
        "single_precision_histogram": True,
        "max_bin": 128,
        "seed": 42,
        'device': 'cuda'
    }

    num_boost_round = trial.suggest_int("num_boost_round", 300, 1500, step=100)
    data_xgb = data.with_columns([
    (pl.col(c).rank("dense") - 1)  # rank starts at 1 → shift to 0
    .fill_null(-1)
    .cast(pl.Int32)
    .alias(c)
    for c in CAT_FEATURES_FINAL
    ])
    hit_at_3 = train_single_split(
        data_xgb=data_xgb,
        params=params,
        num_boost_round=num_boost_round,
        seed=42,
    )

    return hit_at_3  # maximise



d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from pathlib import Path
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, data[:n2]), n_trials=50, show_progress_bar=True, gc_after_trial=True)

# ---------------------------------------------------------------------
# 3) Report best params & save
# ---------------------------------------------------------------------
print("\nBest hit@3:", round(study.best_value, 4))
print("Best params:\n", study.best_params)

out_dir = Path("optuna_results")
out_dir.mkdir(exist_ok=True, parents=True)
(out_dir / "best_params.json").write_text(study.best_trial.params.__repr__())

[I 2025-07-31 12:31:00,571] A new study created in memory with name: no-name-ca9840d4-f07f-47cf-bc7a-bff5524b0287
  0%|          | 0/50 [00:00<?, ?it/s]d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [12:31:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "single_precision_histogram" } are not used.

  self.starting_round = model.num_boosted_rounds()
Best trial: 0. Best value: 0.35665:   2%|▏         | 1/50 [01:58<1:36:35, 118.28s/it]

[I 2025-07-31 12:32:58,824] Trial 0 finished with value: 0.3566495878814881 and parameters: {'eta': 0.016799433682372827, 'max_depth': 5, 'min_child_weight': 20, 'subsample': 0.9614366002961072, 'colsample_bytree': 0.7213201036469383, 'gamma': 1.7137185245609894, 'lambda': 6.041193036497697e-07, 'alpha': 0.00901802221641974, 'num_boost_round': 1500}. Best is trial 0 with value: 0.3566495878814881.


d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [12:33:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "single_precision_histogram" } are not used.

  self.starting_round = model.num_boosted_rounds()
Best trial: 1. Best value: 0.420077:   4%|▍         | 2/50 [03:24<1:19:30, 99.39s/it] 

[I 2025-07-31 12:34:24,991] Trial 1 finished with value: 0.42007704762436493 and parameters: {'eta': 0.08870660003385719, 'max_depth': 7, 'min_child_weight': 17, 'subsample': 0.6373344331189786, 'colsample_bytree': 0.9551034843619213, 'gamma': 1.321789700331823, 'lambda': 1.9735635430171912e-07, 'alpha': 1.868435583931355, 'num_boost_round': 600}. Best is trial 1 with value: 0.42007704762436493.


d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [12:34:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "single_precision_histogram" } are not used.

  self.starting_round = model.num_boosted_rounds()
Best trial: 1. Best value: 0.420077:   6%|▌         | 3/50 [04:42<1:10:08, 89.55s/it]

...

d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:10:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "single_precision_histogram" } are not used.

  self.starting_round = model.num_boosted_rounds()
Best trial: 32. Best value: 0.460573:  66%|██████▌   | 33/50 [41:41<32:00, 112.96s/it]

[I 2025-07-31 13:12:41,855] Trial 32 finished with value: 0.46057327682355564 and parameters: {'eta': 0.16294447195189452, 'max_depth': 8, 'min_child_weight': 20, 'subsample': 0.6484825388288731, 'colsample_bytree': 0.7708348393224855, 'gamma': 0.7165007134041275, 'lambda': 7.5440807588555035e-06, 'alpha': 0.062420940263057806, 'num_boost_round': 1400}. Best is trial 32 with value: 0.46057327682355564.


d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:12:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "single_precision_histogram" } are not used.

  self.starting_round = model.num_boosted_rounds()
Best trial: 32. Best value: 0.460573:  66%|██████▌   | 33/50 [42:54<22:06, 78.01s/it] 


[W 2025-07-31 13:13:54,838] Trial 33 failed with parameters: {'eta': 0.1683192252772687, 'max_depth': 8, 'min_child_weight': 20, 'subsample': 0.650877028538525, 'colsample_bytree': 0.7636606745146671, 'gamma': 0.4982249771945027, 'lambda': 1.715039819604432e-05, 'alpha': 0.036004453144688475, 'num_boost_round': 1400} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\Work\FlightRank 2025 Aeroclub RecSys Cup\.venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\1tiii\AppData\Local\Temp\ipykernel_12180\2573677029.py", line 3, in <lambda>
    study.optimize(lambda trial: objective(trial, data[:n2]), n_trials=50, show_progress_bar=True, gc_after_trial=True)
                                 ~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "C:\Users\1tiii\AppData\Local\Temp\ipykernel_12180\1600262994.py", line 123, in objective
    hit_at_3 = train_single_split(
        data_xgb=data_xg

KeyboardInterrupt: 

In [12]:
study.best_params

{'eta': 0.16294447195189452,
 'max_depth': 8,
 'min_child_weight': 20,
 'subsample': 0.6484825388288731,
 'colsample_bytree': 0.7708348393224855,
 'gamma': 0.7165007134041275,
 'lambda': 7.5440807588555035e-06,
 'alpha': 0.062420940263057806,
 'num_boost_round': 1400}

In [5]:
import numpy as np
import polars as pl
import xgboost as xgb
from copy import deepcopy
from typing import List, Tuple


# ─────────────────────────────────────────────────────────────────────────
# 1) Train / evaluate on a subset of columns
# ─────────────────────────────────────────────────────────────────────────
def train_single_split_subset(
    data_xgb: pl.DataFrame,
    feature_cols: List[str],
    params: dict,
    num_boost_round: int,
    seed: int = 42,
) -> float:
    """Return hit@3 on an 80/20 group‑based split using only feature_cols."""
    rng = np.random.default_rng(seed)
    groups_unique = data_xgb.select(GROUP_COL).unique().to_numpy().ravel()
    val_groups = rng.choice(
        groups_unique, size=max(1, int(0.2 * len(groups_unique))), replace=False
    )

    val_df   = data_xgb.filter(pl.col(GROUP_COL).is_in(val_groups))
    train_df = data_xgb.filter(~pl.col(GROUP_COL).is_in(val_groups))

    dtrain = xgb.DMatrix(
        train_df.select(feature_cols).to_numpy(),
        label=train_df.select(LABEL_COL).to_numpy().ravel(),
        group=get_group_sizes(train_df.select(GROUP_COL).to_numpy().ravel()),
        feature_names=feature_cols,
    )
    dval = xgb.DMatrix(
        val_df.select(feature_cols).to_numpy(),
        label=val_df.select(LABEL_COL).to_numpy().ravel(),
        group=get_group_sizes(val_df.select(GROUP_COL).to_numpy().ravel()),
        feature_names=feature_cols,
    )

    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=[(dval, "val")],
        early_stopping_rounds=30,
        verbose_eval=False,
    )

    # --- hit@3 -----------------------------------------------------------
    preds       = bst.predict(dval)
    ranker_ids  = val_df.select(GROUP_COL).to_numpy().ravel()
    labels      = dval.get_label()

    order = np.argsort(ranker_ids, kind="stable")
    ranker_ids, labels, preds = ranker_ids[order], labels[order], preds[order]

    hit_total = group_total = 0
    i = 0
    while i < len(ranker_ids):
        g = ranker_ids[i]
        j = i
        while j < len(ranker_ids) and ranker_ids[j] == g:
            j += 1
        if (j - i) > 10:                       # skip tiny groups
            top3 = np.argsort(preds[i:j])[::-1][:3]
            hit_total += int(labels[i:j][top3].sum() > 0)
            group_total += 1
        i = j
    return hit_total / group_total if group_total else 0.0


# ─────────────────────────────────────────────────────────────────────────
# 2) Greedy step‑forward selection
# ─────────────────────────────────────────────────────────────────────────
def step_forward_selection(
    data_xgb: pl.DataFrame,
    all_features: List[str],
    params: dict,
    num_boost_round: int,
    seed: int = 42,
    min_improvement: float = 1e-4,
) -> Tuple[List[str], float]:
    """
    Greedy SFS: start with empty set, add best column each round.
    Stops when no remaining feature improves hit@3 by > min_improvement.
    Returns (selected_features, best_score).
    """
    remaining = set(all_features)
    selected: List[str] = []
    best_score = 0.0

    while remaining:
        trial_scores = {}
        for feat in remaining:
            score = train_single_split_subset(
                data_xgb,
                selected + [feat],
                deepcopy(params),          # XGBoost mutates dict → copy!
                num_boost_round,
                seed,
            )
            trial_scores[feat] = score
            
        # pick the feature with the highest score
        feat_best, score_best = max(trial_scores.items(), key=lambda kv: kv[1])

        # check if it is a real improvement
        if score_best - best_score < min_improvement:
            print(f"Stopping: no feature improves hit@3 by ≥{min_improvement:.1e}")
            break

        selected.append(feat_best)
        remaining.remove(feat_best)
        best_score = score_best
        print(f"Added {feat_best:>30}  →  hit@3 = {best_score:.4f}")

    return selected, best_score


In [6]:
params = {
"objective":   "rank:pairwise",
"eval_metric": "ndcg@3",
"tree_method": "hist",      # 🔥 much faster
"seed":        42,
"n_jobs":      -1,
'eta': 0.11258528754406343,
 'max_depth': 7,
 'min_child_weight': 9,
 'subsample': 0.6386911919274852,
 'colsample_bytree': 0.8167665572930206,
 'gamma': 3.3978723412953182,
 'lambda': 0.0015573270572558455,
 'alpha': 0.0007046464277948389,
 'device': 'cuda'}

In [7]:
data_ready = data.with_columns([
(pl.col(c).rank("dense") - 1)  # rank starts at 1 → shift to 0
.fill_null(-1)
.cast(pl.Int32)
.alias(c)
for c in cat_features_final
])
chosen_feats, final_hit3 = step_forward_selection(
     data_xgb         = data_ready,
     all_features     = feature_list,
     params           = params,
     num_boost_round  = 1000,
     seed             = 42,
 )

Added               opt_ticket_score  →  hit@3 = 0.5022
Added      miniRules1_monetaryAmount  →  hit@3 = 0.5263
Added               avg_oneway_price  →  hit@3 = 0.5383
Added                      log_price  →  hit@3 = 0.5562
Added                   company_freq  →  hit@3 = 0.5749
Added       legs0_departureAt_period  →  hit@3 = 0.5813
Added       legs1_departureAt_period  →  hit@3 = 0.5962


KeyboardInterrupt: 